In [2]:
import sys

In [3]:
print("\n".join(sys.path))
sys.path.append("/development/lingvo-source")
sys.path.append("/development/lingvo-source/galvasr2/align")

/tmp/spark-7ae2d8f9-803b-4324-9414-5602094809ac/userFiles-df1331d6-878d-4ca9-8360-955ba2e66e56
/development/lingvo-source/galvasr2
/install/spark/python/lib/py4j-0.10.9-src.zip
/install/spark/python
/development/lingvo-source/galvasr2
/install/miniconda3/envs/100k-hours-lingvo-3/lib/python37.zip
/install/miniconda3/envs/100k-hours-lingvo-3/lib/python3.7
/install/miniconda3/envs/100k-hours-lingvo-3/lib/python3.7/lib-dynload

/install/miniconda3/envs/100k-hours-lingvo-3/lib/python3.7/site-packages
/install/miniconda3/envs/100k-hours-lingvo-3/lib/python3.7/site-packages/pyspark-3.0.0-py3.7.egg
/install/miniconda3/envs/100k-hours-lingvo-3/lib/python3.7/site-packages/py4j-0.10.9-py3.7.egg
/install/miniconda3/envs/100k-hours-lingvo-3/lib/python3.7/site-packages/IPython/extensions
/root/.ipython


In [4]:
! ls /development/lingvo-source/galvasr2

AlignmentExploration.ipynb  language_histogram.py
BUILD			    language_histogram.py~
BUILD~			    launch_pyspark_notebook.sh
align			    launch_pyspark_notebook.sh~
galvasr_latgen_faster.py~   schema.py
galvasr_tokenize_words.py~  schema.py~


In [9]:
from galvasr2.align.spark.schemas import ARCHIVE_ORG_SCHEMA
from pyspark.sql import functions as F

df = spark.read.format('json').schema(ARCHIVE_ORG_SCHEMA).load("../ALL_CAPTIONED_DATA.jsonl.gz")

In [77]:
exploded_df = df.withColumn("exploded_files", F.explode(df.files))
#exploded_df.printSchema()
text_df = exploded_df.select(
    exploded_df.identifier,
    exploded_df.exploded_files["name"].alias("text_document_id"),
    exploded_df.exploded_files.format.alias("text_document_format")).where(
    (
    (exploded_df.exploded_files.format == 'SubRip') | 
    (exploded_df.exploded_files.format == 'Web Video Text Tracks') |
    (exploded_df.exploded_files.format == "Closed Caption Text")
    )
    &
    # isNull() indicates that this file was not derived from any other file
    # exploded_df.exploded_files.source == 'original' is probably equivalent
    (exploded_df.exploded_files.original.isNull())
    # According to https://archive.org/help/derivatives.php, "Closed Caption Text" is automatically converted to SubRip format.
    # (exploded_df.exploded_files.format == "Closed Caption Text")
)
# 
#    exploded_df.exploded_files.format == "Web Video Text Tracks" | 
#    exploded_df.exploded_files.format == "Closed Caption Text")
audio_df = exploded_df.select(exploded_df.identifier,
                              exploded_df.exploded_files["name"].alias("audio_document_id")).where(exploded_df.exploded_files.format == 'MP3')

joined_df = audio_df.join(text_df, "identifier")
joined_df.printSchema()
print(joined_df.count())
return
#joined_df = joined_df.select(F.lit("archive.org").alias("source"), 
#                             joined_df.identifier,
#                             joined_df.exploded_files["name"].alias("audio_document_id"),
#                             )
print(blah.head(10))

print(text_df.count())
# 19487
# 19866 without the isNull() constraint. Rather fishy, isn't it?
# text_df.printSchema()
print(text_df.where(text_df["`exploded_files.format`"] == "SubRip").count())
print(text_df.where(text_df["`exploded_files.format`"] == "Web Video Text Tracks").count())
print(text_df.where(text_df["`exploded_files.format`"] == "Closed Caption Text").count())

print(audio_df.count())
# 15026

root
 |-- identifier: string (nullable = true)
 |-- audio_document_id: string (nullable = true)
 |-- text_document_id: string (nullable = true)
 |-- text_document_format: string (nullable = true)

19615


SyntaxError: 'return' outside function (<ipython-input-77-9e32b44e1061>, line 27)

In [47]:
"""
[Row(exploded_files.name='10_10_2017_Essex_Junction_Trustees.en.vtt'),
 Row(exploded_files.name='10_10_2017_Essex_Junction_Trustees.es.vtt'),
 Row(exploded_files.name='10_11_2017_Burlington_Telecom_Advisory_Board.en.vtt'),
 Row(exploded_files.name='10_11_2017_Burlington_Telecom_Advisory_Board.es.vtt'),
 Row(exploded_files.name='10_12_2017_Essex_Town_Planning_Commission.en.vtt'),
 Row(exploded_files.name='10_12_2017_Essex_Town_Planning_Commission.es.vtt'),
 Row(exploded_files.name='10_16_2017_Essex_Town_Selectboard.en.vtt'),
 Row(exploded_files.name='10_16_2017_Essex_Town_Selectboard.es.vtt'),
 Row(exploded_files.name='10_16_2017_Winooski_City_Council.en.vtt'),
 Row(exploded_files.name='10_16_2017_Winooski_City_Council.es.vtt')]
"""

text_df.printSchema()

text_df.filter(text_df["`exploded_files.name`"].contains("10_10_2017_Essex_Junction_Trustees")).head(10)

root
 |-- exploded_files.name: string (nullable = true)



[Row(exploded_files.name='10_10_2017_Essex_Junction_Trustees.asr.srt'),
 Row(exploded_files.name='10_10_2017_Essex_Junction_Trustees.en.vtt'),
 Row(exploded_files.name='10_10_2017_Essex_Junction_Trustees.es.vtt')]

In [54]:
exploded_df.groupBy(exploded_df.exploded_files.original).count().orderBy('count', ascending=False).show()

+------------------------+------+
|exploded_files[original]| count|
+------------------------+------+
|                    null|148577|
|    PlanningCommissio...|   502|
|    BC-2016-0823-Plan...|   500|
|    BC_PlanningCommis...|   462|
|    town-meeting-2_05...|   386|
|    CC-2016-0509-Gove...|   337|
|          hbc070213v.asf|   318|
|            Grad2002.mpg|   318|
|    hrs13REF2154_1002...|   304|
|     BOS_092517-CL19.mpg|   304|
|            BOSNov12.mpg|   304|
|    mv_senate_proceed...|   304|
|    1_2_2018_South_Bu...|   304|
|    Richmond_Selectbo...|   304|
|    hrs03A_S2118_0901...|   304|
|    Town Meeting 0606...|   304|
|    Richmond_Selectbd...|   304|
|    hrs02A_S_HVC-210_...|   304|
|    mv_senate_proceed...|   303|
|    mv_house_proceedi...|   303|
+------------------------+------+
only showing top 20 rows



In [1]:
# WEBVTT
# Kind: captions
# Language: es

# Useful WEBVTT metadata. It knows its own language and whether or not these are captions.

import sys
print("\n".join(sys.path))
sys.path.append("/development/lingvo-source")
sys.path.append("/development/lingvo-source/galvasr2/align")

from galvasr2.align.spark import align # import load_transcripts
# import importlib
# importlib.reload("align")
# importlib.reload(sys.modules['galvasr2'])
from galvasr2.align.spark.schemas import ARCHIVE_ORG_SCHEMA
from pyspark.sql import functions as F
from pyspark.sql import types as T

df = spark.read.format('json').schema(ARCHIVE_ORG_SCHEMA).load("../ALL_CAPTIONED_DATA.jsonl.gz")

exploded_df = df.withColumn("exploded_files", F.explode(df.files))


filtered_exploded_df = exploded_df.where(
    # When a file's size is 0 bytes, scripts/archive.org/download_items.py does 
    # not download that file. We therefore filter out size 0 bytes to prevent 
    # file-not-found errors in aling.py::load_transcripts()
    (exploded_df.exploded_files.size.cast(T.LongType()) != 0)
    &
    # This indicates that the file is not "private". 
    # As far as I can tell, the "private" field is either "true" or null. 
    # Trying to read this data as booleans turns every field null for some 
    # reason, so it is currently a String field.
    # Private data is not downloadable by the general public.
    exploded_df.exploded_files.private.isNull()
)

text_df = filtered_exploded_df.select(
    exploded_df.identifier,
    exploded_df.exploded_files["name"].alias("text_document_id"),
    exploded_df.exploded_files.format.alias("text_document_format")).where(
    (exploded_df.exploded_files.format == 'SubRip')
)

audio_df = filtered_exploded_df.select(exploded_df.identifier,
                              exploded_df.exploded_files["name"].alias("audio_document_id")).where(
    (exploded_df.exploded_files.format == 'MP3')
)

joined_df = audio_df.join(text_df, "identifier")

rows = joined_df.select("identifier", "text_document_id", "text_document_format").collect()
# 18349
print("GALVEZ:", len(rows))
#rows = rows[:10]
#print("\n".join(str(row) for row in rows))
#spark.sessionState.config.setConf("spark.sql.files.ignoreMissingFiles", "true")
result = align.load_transcripts(spark, "gs://the-peoples-speech-west-europe/archive_org/Nov_6_2020/ALL_CAPTIONED_DATA", rows)

/tmp/spark-e2e7a8a3-2227-4a4b-a334-4d60d7b1b6d1/userFiles-dfce5c2b-0b23-479a-ae81-6571a821bfc9
/development/lingvo-source/galvasr2
/install/spark/python/lib/py4j-0.10.9-src.zip
/install/spark/python
/development/lingvo-source/galvasr2
/install/miniconda3/envs/100k-hours-lingvo-3/lib/python37.zip
/install/miniconda3/envs/100k-hours-lingvo-3/lib/python3.7
/install/miniconda3/envs/100k-hours-lingvo-3/lib/python3.7/lib-dynload

/install/miniconda3/envs/100k-hours-lingvo-3/lib/python3.7/site-packages
/install/miniconda3/envs/100k-hours-lingvo-3/lib/python3.7/site-packages/pyspark-3.0.0-py3.7.egg
/install/miniconda3/envs/100k-hours-lingvo-3/lib/python3.7/site-packages/py4j-0.10.9-py3.7.egg
/install/miniconda3/envs/100k-hours-lingvo-3/lib/python3.7/site-packages/IPython/extensions
/root/.ipython
GALVEZ: 17956


In [2]:
a = result.head()
print(a)

Row(transcript="I. Know. That. Good morning this meeting in the budget Finance Committee will please come to order . Members we've got another full day ahead of us. Looks like we've got a lot of testified to this morning with happy to see and I'd like to start the morning by introducing the members present today of starting with but Carol. Good morning guys Spicer. Good morning Kelly Cochran. Good morning Don couch morning chair long time no see yeah. Stacey Carrillo nothing. And excuse for an hour the chair of the committee Ricky hall comma Don Goodman and Mike victory no Mr victory no as in Honolulu scorning with legislative issues that all of us should be very concerned about. The well I won't get into that we've got lots of testimony. Missed by so thank you very much because Mr victory you know he's representing us this morning I'm going to channel him. Happy to everyone thank you very much thank you for that reminder and I was reminded of that in the elevator as he as you know all

In [6]:
result.limit(4).write.mode("overwrite").format("json").save("/home/ws15dgalvez/dumpblahblah.json")

In [3]:
exploded_df.groupBy(exploded_df.exploded_files.private).count().show()

+-----------------------+-------+
|exploded_files[private]|  count|
+-----------------------+-------+
|                   null|1352804|
|                   true|    294|
+-----------------------+-------+



In [15]:
exploded_df.where(exploded_df.exploded_files.name == "0084_Tomorrow_Always_Comes_20_00_32_00_3mb.asr.srt").count()

TypeError: condition should be string or Column

In [5]:
import sys
sys.path.append("/development/lingvo-source")
sys.path.append("/development/lingvo-source/galvasr2/align")

from galvasr2.align.spark.align import prepare_generate_lm_udf

generate_lm_udf = prepare_generate_lm_udf("/install/kenlm/build/bin/", "/development/lingvo-source/tmpworkdir")

df = spark.read.format("json").load("/home/ws15dgalvez/dump")
rows = df.select(generate_lm_udf(df.transcript)).head(1)
print(rows)

PythonException: 
  An exception was thrown from Python worker in the executor. The below is the Python worker stacktrace.
Traceback (most recent call last):
  File "/install/spark/python/lib/pyspark.zip/pyspark/worker.py", line 589, in main
    func, profiler, deserializer, serializer = read_udfs(pickleSer, infile, eval_type)
  File "/install/spark/python/lib/pyspark.zip/pyspark/worker.py", line 447, in read_udfs
    udfs.append(read_single_udf(pickleSer, infile, eval_type, runner_conf, udf_index=i))
  File "/install/spark/python/lib/pyspark.zip/pyspark/worker.py", line 254, in read_single_udf
    f, return_type = read_command(pickleSer, infile)
  File "/install/spark/python/lib/pyspark.zip/pyspark/worker.py", line 74, in read_command
    command = serializer._read_with_length(file)
  File "/install/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 172, in _read_with_length
    return self.loads(obj)
  File "/install/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 

In [6]:
df = spark.read.format('tfrecord').option("recordType", "Example").load("gs://the-peoples-speech-west-europe/forced-aligner/vad-segments-dump/Mar_13_2021/small_dataset_better_model/vad_pcm_tfrecords")

In [8]:
rows = df.head(1)

In [18]:
import numpy as np
import sys
np.set_printoptions(threshold=sys.maxsize)
np.max(np.abs(np.asarray(rows[0].frames)))
len(np.asarray(rows[0].frames)) / 16000.

15.81

In [20]:
rows[1].num_utterances_in_audio_document

IndexError: list index out of range

In [1]:
vad_out_dir = "gs://the-peoples-speech-west-europe/forced-aligner/Mar_18_2021/vad_pcm_tfrecords"
print(spark.read.format("tfrecord").option("recordType", "Example").load(vad_out_dir).count())

11175308


In [ ]:
11175308
   18169

In [7]:
import sys
print("\n".join(sys.path))
sys.path.append("/development/lingvo-source")
sys.path.append("/development/lingvo-source/galvasr2/align")
from galvasr2.align.spark.schemas import ARCHIVE_ORG_SCHEMA
from pyspark.sql import functions as F
import pyspark.sql.types as T

def load_audio_id_text_id_mapping(spark, input_catalogue_path: str):
  df = spark.read.format('json').schema(ARCHIVE_ORG_SCHEMA).load(input_catalogue_path)

  exploded_df = df.withColumn("exploded_files", F.explode(df.files))

  filtered_exploded_df = exploded_df.where(
    # When a file's size is 0 bytes, scripts/archive.org/download_items.py does                                                                                                                             
    # not download that file. We therefore filter out size 0 bytes to prevent                                                                                                                               
    # file-not-found errors in aling.py::load_transcripts()                                                                                                                                                 
    (exploded_df.exploded_files.size.cast(T.LongType()) != 0)
    &
    # This indicates that the file is not "private".                                                                                                                                                        
    # As far as I can tell, the "private" field is either "true" or null.                                                                                                                                   
    # Trying to read this data as booleans turns every field null for some                                                                                                                                  
    # reason, so it is currently a String field.                                                                                                                                                            
    # Private data is not downloadable by the general public.                                                                                                                                               
    exploded_df.exploded_files.private.isNull()
    #&
    # "[" and "]" are wild card characters. GCS has very poor support                                                                                                                                       
    # for these. Namely, you can write them but not read them back. More                                                                                                                                    
    # resources here: https://github.com/galv/lingvo-copy/issues/18                                                                                                                                         
    # I simply filter out any files containing these characters for now.                                                                                                                                    
    #(~((exploded_df.identifier.contains("[")) |
    #   (exploded_df.identifier.contains("]"))))
    #&
    #(~((exploded_df.exploded_files["name"].contains("[")) |
    #   (exploded_df.exploded_files["name"].contains("]"))))
  )

  text_df = filtered_exploded_df.select(
    exploded_df.identifier,
    exploded_df.exploded_files["name"].alias("text_document_id"),
    exploded_df.exploded_files.format.alias("text_document_format")).where(
      (exploded_df.exploded_files.format == 'SubRip')
    )

  audio_df = filtered_exploded_df.select(
    exploded_df.identifier,
    exploded_df.exploded_files["name"].alias("audio_document_id")).where(
      (exploded_df.exploded_files.format == 'MP3')
      &
      # https://ia802901.us.archive.org/4/items/disneychannelourhourafterhourafterhourprankstermarathonapril12004/disneychannelourhourafterhourafterhourprankstermarathonapril12004_files.xml               
      (exploded_df.exploded_files["name"].endswith('.mp3'))
    )

  joined_df = audio_df.join(text_df, "identifier")

  return joined_df
df = load_audio_id_text_id_mapping(spark, "gs://the-peoples-speech-west-europe/archive_org/Mar_7_2021/EXPANDED_LICENSES_FILTERED_ACCESS.jsonl.gz")
print(df.count())
# 18177
# 18180

/tmp/spark-522eb431-983c-46c8-9fa8-5d7551fc8eea/userFiles-8dd8223a-0cb4-4cdb-9365-72622e843dfc
/development/lingvo-source/galvasr2
/install/spark/python/lib/py4j-0.10.9-src.zip
/install/spark/python
/development/lingvo-source/galvasr2
/install/miniconda3/envs/100k-hours-lingvo-3/lib/python37.zip
/install/miniconda3/envs/100k-hours-lingvo-3/lib/python3.7
/install/miniconda3/envs/100k-hours-lingvo-3/lib/python3.7/lib-dynload

/install/miniconda3/envs/100k-hours-lingvo-3/lib/python3.7/site-packages
/install/miniconda3/envs/100k-hours-lingvo-3/lib/python3.7/site-packages/pyspark-3.0.0-py3.7.egg
/install/miniconda3/envs/100k-hours-lingvo-3/lib/python3.7/site-packages/py4j-0.10.9-py3.7.egg
/install/miniconda3/envs/100k-hours-lingvo-3/lib/python3.7/site-packages/IPython/extensions
/root/.ipython
/development/lingvo-source
/development/lingvo-source/galvasr2/align
/development/lingvo-source
/development/lingvo-source/galvasr2/align
/development/lingvo-source
/development/lingvo-source/galvasr2

In [14]:
vad_out_dir = "gs://the-peoples-speech-west-europe/forced-aligner/Mar_18_2021/vad_pcm_tfrecords"
df = spark.read.format("tfrecord").option("recordType", "Example").load(vad_out_dir)

rows = df.select(F.sum(F.length(df.frames))).collect()
print(rows)

[Row(sum(length(frames))=1756846505709)]


In [15]:
rows

[Row(sum(length(frames))=1756846505709)]

In [16]:
1756846505709 / 2 / 16_000 / 60 / 60

15250.403695390625